In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2025-05-11 12:37:44--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2744:9400:b:20a5:b140:21, 2600:9000:2744:c800:b:20a5:b140:21, 2600:9000:2744:5000:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2744:9400:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M  10.1MB/s    in 4.5s    

2025-05-11 12:37:49 (10.1 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]

--2025-05-11 12:37:49--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2744:a000:b:20a5:b140:21, 2600:9000:2744:5000:b:20a5:b140:21

In [3]:
df_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
# Q1 - How many columns are in the January data? 
print(f"There are {len(df_jan.columns)} columns.") 

There are 19 columns.


In [5]:
# Q2 What's the standard deviation of the trips duration in January?
df_jan["duration"] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime)
df_jan["duration"] = df_jan["duration"].apply(lambda td: td.total_seconds() / 60)
print(f"The standard deviation of trips in January is {df_jan["duration"].std():.2f} minutes.")


The standard deviation of trips in January is 42.59 minutes.


In [6]:
# Q3 What's the fraction of records left after dropping outliers?
df_jan_out = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60 )]
print(f"After removing outliers we are left with {len(df_jan_out) / len(df_jan)*100:.0f}% of the records.")

After removing outliers we are left with 98% of the records.


In [15]:
# Q4 - OneHot Encoding
dv = DictVectorizer()
categorical = ["PULocationID", "DOLocationID"]

df_jan_out.loc[:, categorical] = df_jan_out[categorical].astype(str).copy()

train_dicts = df_jan_out[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

print(f"The dimensionality of the training matrix is {X_train.shape[1]}")

The dimensionality of the training matrix is 515


In [ ]:
# Q5 - Training the model. RMSE on train
model = LinearRegression()
y_train = df_jan_out["duration"].values
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
rmse_train = root_mean_squared_error(y_train, y_pred)
print(f"The RMSE on train data is: {rmse_train:.2f}")

The RMSE on train data is: 7.65


In [18]:
# Q6 - Evaluating the model. RMSE on valiudtion
df_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')
df_feb["duration"] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime)
df_feb["duration"] = df_feb["duration"].apply(lambda td: td.total_seconds() / 60)
df_feb = df_feb[(df_feb["duration"] >= 1) & (df_feb["duration"] <= 60 )]

df_feb.loc[:, categorical] = df_feb[categorical].astype(str)

val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_feb["duration"].values

y_pred_val = model.predict(X_val)
rmse_val = root_mean_squared_error(y_val, y_pred_val)
print(f"The RMSE on validation data is: {rmse_val:.2f}")

/tmp/ipykernel_3097140/1854304750.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_feb.loc[:, categorical] = df_feb[categorical].astype(str)
/tmp/ipykernel_3097140/1854304750.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '26' '145' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_feb.loc[:, categorical] = df_feb[categorical].astype(str)


The RMSE on validation data is: 7.81
